0 Clustering: The K-means Clustering (continued)
-------------------

In this session, you are going to get yourself familiar with the **unsupervised** classification. It is named as **unsupervised** because there is no **label** in the dataset indicating the pattern of classes within the dataset. And the classification results solely depend on how the classification algorithms explore the dataset. For instance, data points adjacent to each other can be treated as from one class. Even for such simple formalization, there can already be many ways to characterize the adjacency. The simplest way is to measure the Euclidean distance between the data points in the feature space and **cluster** the adjacent data points into classes. The **K-means** is an algorithm can help you to achieve this. The **K_means** is thus used as the starting point of **unsupervised** classification. Apart from the **K-means**, there are also **Gaussian Mixture** models that try to impose multi-modal *Gaussian* distributions upon the data points in the feature space, which also **cluster** the data points into classes according in terms of probability. For your information, these few algorithms including the **K-means** and **Gaussian Mixture** all fall into the **expectation maximization** (**EM**) family of algorithms. So, if you are interested in them, please do explore further background information about the **EM**s. However, it is not possible to cover all of them in this session. More details can be found in **[Pattern Recognition and Machine learning](http://users.isr.ist.utl.pt/~wurmd/Livros/school/Bishop%20-%20Pattern%20Recognition%20And%20Machine%20Learning%20-%20Springer%20%202006.pdf)**, Chapter 9, by Chris Bishop.

You will be using the GIS and Remote Sensing dataset of real world land use and land cover (LULC) types in the Netherlands for the practice of **unsupervised** clustering. But to get started, you will first look at some artificially generated data, which I call the dummy dataset. It is always quite advantageous to use dummy dataset. First, you can simulate different situations. And then, you know the nature of the dataset and it will be easy for you to evaluate the performance of algorithms, such as the **K-means**, in capturing that nature. After going through the rationale of the **K-means**, you will work on your GIS and Remote Sensing dataset to evaluate the performance of the algorithms on real dataset. The structure of this session can be summarized as:

- 0.0 K-means on dummy dataset
- 0.1 K-means through existed modules in Sklearn
- 0.2 The K-means on real world GIS and Remote Sensing datasets
- 0.3 Accuracy assessment

In this notebook, the last two sections are involved. The real world remote sensing datasets involve pixel values from multiple spectral bands, which means each pixel is characterized by multiple spectral values, thus the **K-means** classifies the pixels in a feature space with higher dimensions.

<font color='blue'> ***Throughout the notebook, you will encounter Questions, highlighted in blue. Please feel free to discuss them with your classmates and advisors***</font>

<font color='green'> ***Throughout the notebook, you will also encounter shorter and longer Exercises, highlighted in green.***</font>

In another notebook, you would see the first two sections, which are based on a dummy dataset with two dimensions. This means the **K-means** will be deployed in a 2-dimensional feature space. The first section depends only on the most frequenlty used modules of ***Numpy*** and ***Matplotlib***, hence, you won't bother with many modules in understanding the basics. Thus please feel free to check the more basic contents if you would like to explore more details with the dummy dataset.

### 0.2 The K-means on real world GIS and Remote Sensing datasets.

Now, it's time to try the **K-means** on real world dataset! In this section, you are going to classify the land use and land cover (LULC) types in the Netherlands by using satellite imagery. Essentially, you are going to classify the image pixels according to their spectral signature assuming that the LULC types are manifested in the pixel spectral signature. The pixel spectral signature is measured by multiple image bands, each of which measuring a spectral value for the pixels. Given the multiple bands, each pixel is measured in multiple values, thus each pixel can be characterized in a multi-dimensional feature space! The **K-means** will be applied in such a multi-dimensional feature space to cluster the pixels so that the clusters reflect the LULC types.

Expectedly, the more different bands we have the better may the pixels be differentiated. But we will start with two bands for the ease of visualization and inspection as we did for the dummy dataset. We will also start with a small dataset, in which few sample areas in the Netherlands with manually delineated LULC types are involved. Hence, in the beginning, the rationale of the experiment with this real world dataset is same as you played with the dummy dataset -- you already know the true classes, but you need to explore how the **K-means**, the **unsupervised** algorithm, resembles the classes. This can be treated as a simply way to evaluate the performance of the **K-means**.

Later, you will be classify the LULC of the whole country, and play with accuracy assessment more seriously as well. The feature space, which you will deploy your classification is totally determined by yourself by choosing different band combination.

In [ ]:
# Before reading the data we need to first clone the data on Github to our Colab workspace
!git clone https://github.com/jonwangio/uu_ml

The first thing you'd like to do is to get familiar with the dataset in the study area. The dataset has already been used in the previous sessions of the Applied Data Science. In this section, you will use few modules to load, process and visualize the data: ***rasterio***, ***geopandas***, ***sklearn***, ***numpy***, ***seaborn*** and ***matplotlib***.

In [ ]:
import matplotlib.pyplot as plt
% pip install seaborn
import seaborn as sns
import numpy as np
% pip install geopandas
import geopandas as gpd
% pip install rasterio
import rasterio
from rasterio.plot import show
from sklearn import cluster

# We first test the K-means in places with manually delineated LULC.
# The data contains sample LULC areas around dutch provinces North Holland and Utrecht.
# The manually labelled LULC types will be compared with clustering results from the K-means.
aoi = gpd.read_file('uu_ml/data/aoi_NL_5_classes.shp')

print (aoi.head())

In [ ]:
# Below is a visualization of the sample LULC areas superimposed on a satellite image of part of the Netherlands
file_location = 'uu_ml/data/b5_2015.TIF'
b5_2020 = rasterio.open(file_location, nodata=0)

# We also prepare the color codes for visualization
colors = [(257, 71, 27), (98, 93, 78), (14, 79, 58), (26, 0, 50), (75, 90, 85), (347, 72, 60), (246, 79, 60)]
cols = []
for col in colors:
    pal = sns.light_palette(col, input="husl", n_colors=4)
    for rgb in pal[1:]:
        cols.append(rgb)

# A preview of color codes. Please delete the triple quotation marks to run the code.

fig, ax = plt.subplots(figsize=(20, 5))
for i, c in enumerate(cols):
    ax.add_artist(plt.Circle((i, 0), 0.4, color=c))
    plt.text(i, -1, i, horizontalalignment='center')
    ax.set_axis_off()
    ax.set_aspect(1)
    ax.autoscale()
    plt.xlim(-1.25,43.25)
    plt.ylim(-1,1)


# Assign color codes to LULC types 
symbology = {'Agriculture': cols[5],
             'Clear water': cols[20],
             'Deciduous forest': cols[3],
             'Residential': cols[17],
             'Sand': cols[11]}

# Visualize
fig,ax = plt.subplots(1,1, figsize=(10,10))
show(b5_2020, ax=ax, cmap='gray', alpha=0.25)
aoi.plot(ax=ax, column='land_cover', legend=True, color=aoi['land_cover'].map(symbology))

from matplotlib.lines import Line2D
custom_points = [Line2D([0], [0], marker="o", linestyle="none", markersize=5, color=color) for color in symbology.values()]
leg_points = ax.legend(custom_points, symbology.keys(), loc='upper right', frameon=False)
ax.add_artist(leg_points)

These manually delineated/classified sample LULC types are already prepared in a *.csv* file. You can test the **K-means** on these sample LULC areas after loading this file.

In [ ]:
import pandas as pd
lulc = pd.read_csv('uu_ml/data/stack_aoi_2015.csv')

# View some sample rows of the data
lulc.head()

As you saw that there are 7 bands involved in this dataset, thus each of the pixels is measured by 7 spectral values, and can potentially be characterized in a feature space as high as 7 dimensions. But as mentioned above, lower dimensions would be helpful to visualize the data. We will then visualize the dataset with different band combination to see how the patterns of LULC classes represented in feature space of 2 dimensions.

We will again use the ***seaborn*** module to conduct the visual comparison.

In [ ]:
# As you can see, we are using a subset of the dataset for "speed" plotting
lulc_sub = lulc.sample(n=500)

# Adjust aoi numbers to land cover
aoi['aoi_cat'] = pd.Categorical(aoi['class'])
classes = aoi.loc[:,['land_cover', 'aoi_cat']].drop_duplicates()
class_dict = {k:v for k,v in zip(classes.aoi_cat, classes.land_cover)}
lulc_sub.replace({'aoi': class_dict}, inplace=True)
sns.pairplot(lulc_sub.loc[:,['B3', 'B4', 'B5', 'B6', 'aoi']], 
             hue='aoi', palette=symbology, diag_kind="hist", 
             corner=True, 
             plot_kws=dict(linewidth=0.5, alpha=0.5))


<font color='blue'>***Question 0.1:Visually, how are the data points of known LULC types separated in the feature space defined by different band combinations?***</font>

Please note that the ***seaborn.pairplot*** is diagonally symmetric. In the next few cells, you will be experimenting with **K-means** on different band combinations that you think are reasonable.

For example, in the figure above, it seems that band 5 together with anyone of the other bands can separate the 5 LULC types reasonably well. Then let's start with band 5 with another band, say band 6, for a demonstration. To test if the **K-means** will work properly, you can again apply the ***sklearn*** module. Now, let's see if the **K-means** captures the pattern of the classes above.

In [ ]:
from sklearn import cluster

# Recall that you have to decide the number of clusters.
# But also keep in mind that it is not always easy to make the decision as you would in this case with nicely chosen data.
kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B5'].values, lulc_sub['B6'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data

# Try to inspect what exactly contained in the results. For instance, how the 'labels' are created and represented?

# Now all the data points are labelled, you can again visualize the clusters by using the 'labels' just as you did with the 'closest_ind'.
plt.figure(figsize=(8,8))
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8, label='Sample LULC clusters through K-means')
plt.title('LULC in B5 and B6')
plt.legend()

# Compare the results with those you obtained above.

How about all the other band combinations? Is it always good with a combination of two bands?

In [ ]:
fig = plt.figure(figsize=(15,10))
lb = 'Sample LULC clusters through K-means'

kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B3'].values, lulc_sub['B4'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data
plt.subplot(2, 3, 1)
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B3 and B4')
#plt.legend()

kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B3'].values, lulc_sub['B5'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data
plt.subplot(2, 3, 2)
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B3 and B5')

kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B3'].values, lulc_sub['B6'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data
plt.subplot(2, 3, 3)
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B3 and B6')

kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B4'].values, lulc_sub['B5'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data
plt.subplot(2, 3, 4)
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B4 and B5')

kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B4'].values, lulc_sub['B6'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data
plt.subplot(2, 3, 5)
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B4 and B6')

kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc_sub['B5'].values, lulc_sub['B6'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data
plt.subplot(2, 3, 6)
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B5 and B6')


<font color='blue'>***Question 0.2: Which band combination shows a clustering in the feature space that coincides well with the chosen number of five clusters?***</font>.

Please recall, even for these sample areas, we have only used a subset (500 data points) for the above experiments. Why not to try the **K-means** over the entire dataset of the delineated LULC types? Please go ahead. If some of the bands combination, for instance band 5 and 6, seems good to apply the **K-means** for unsupervised classification, we can of course continue with that.

In [ ]:
kmean_cluster = cluster.KMeans(n_clusters=5)  # This is a function to be fitted to the data
points = np.c_[lulc['B5'].values, lulc['B6'].values]
labels = kmean_cluster.fit_predict(points)  # Fitting the data

# Try to inspect what exactly contained in the results. For instance, what is the 'kmean_cluster'?

# Now all the data points are labelled, you can again visualize the clusters by using the 'labels' just as you did with the 'closest_ind'.
plt.figure(figsize=(8,8))
plt.scatter(points[:, 0], points[:, 1], s=40, c=labels, cmap='Paired', marker='o', alpha=0.8)
plt.title('LULC clusters in B5 and B6')
#plt.legend()

You may have walked through the first part of this session, where you tried to inspect in details about the update process of the centroid. If not, let's repeat the procedure on this LULC datasets. Please try different number of centroids to visualize how *proper/improper* number of centroids will lead to different results. 

By default **K-Means** clustering has a maximum of 300 iterations. The functions below are written specifically to emulate what is actually going on within the **K-means** functionality provided by the ***sklearn***. Please try different number of iteration by changing the ***range*** of the ***for loops*** to see how the centroids move throughout different number of iterations.

In [ ]:
# Few functions to capture the update of the cluster centroids.

def initialize_centroids(points, k):
    """returns k centroids from the initial points"""
    centroids = points.copy()
    np.random.shuffle(centroids)
    return centroids[:k]

def closest_centroid(points, centroids):
    """returns an array containing the index to the nearest centroid for each point"""
    distances = np.sqrt(((points - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

def move_centroids(points, closest, centroids):
    """returns the new centroids assigned from the points closest to them"""
    return np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])

In [ ]:
# Try to call your function, and inspect how centroids are updated.
centroids = initialize_centroids(points, 3)

# We can of course visualize how centroids are updated throughout the process.
fig = plt.figure(figsize=(15,15))
for i in range(9):
    closest_ind = closest_centroid(points, centroids)
    centroids = move_centroids(points, closest_ind, centroids)
    plt.subplot(3, 3, i+1)
    plt.scatter(points[:, 0], points[:, 1], s=40, c=closest_ind, cmap='Paired', marker='x', label='training data')
    plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)

<font color='blue'>***Question 0.3: Of course we've already known the actual number of LULC types. But as an unsupervised learning algorithm, you do not expect this to happen quite often. How difficult do you think to determine the number of clusters in the case above?***</font>.

Now you may wonder how does the **K-means** perform in these few senarios compared to the manually delineated LULC types? In the next section, you will be evaluate the performance through accuracy assessment.

### 0.3 Accuracy assessment.

In this section, we will be evaluate how **K-means** generated clusters capture the pattern of known LULC types. Unlike **supervised** learning where you can always explicitly train your model/algorithm with known targets, as well as evaluate (validate/test) the trained model/algorithm, **unsupervised** learning algorithms such as the **K-means** cannot always be evaluated easily. More often, **unsupervised** learning are most used for exploring the patterns within a given dataset. But here, in our case of mapping LULC in the Netherlands, we can take the manually delineated LULC samples as reliable targets, and use them to do some simple accuracy assessment.

We will first evaluate the performance of the **K-means** on the LULC samples. Then we will apply the **K-means** to larger areas, for instance an area covering almost the entire Netherlands, and see if the automatically generated clusters out of the large dataset would still capture the LULC types in our sample areas.

In order to assess the accuracy, we first need to make the automatically generated clusters and the known LULC types comparable. This means that we need to have an idea about which LULC type is captured by each of the clusters. This further means that we need to interpret the clusters before comparison. Interpretation can of course achieved through visualization, for instance, to see how automatically generated clusters overlap with the sample LULC types. 

Quantitatively, interpreting and comparing automatically generated clusters can be achieved by using the **confusion matrix**, where manually labelled data points can be compared with automatically generated ones. In this case, we will check how a number of pixels in each **K-means** generated cluster will be spreaded over different manually labelled LULC types. Many other more advanced statistics can be derived to assess the performance of the **K-means** as you will see.

In [ ]:
# Confusion matrix can be generated by using pandas.

import pandas as pd

y_actu = pd.Series(list(lulc['aoi'].values), name='Manual delineation')
y_pred = pd.Series(list(labels), name='K-means')
df_confusion = pd.crosstab(y_actu, y_pred)

# Show the confusion matrix
df_confusion

In the **confusion matrix**, the clusters are labelled as integers, and the manually delineated LULC types are encoded as: *11 (clear water)*, *21 (sand)*, *31 (residential)*, *52 (agriculture)*, and *61 (deciduous forest)*.

How would you interpret the matrix, for instance, column-wise and row-wise? In this case, for each of the columns, it shows how a specific automatically generated cluster captures many of the labelled LULC types. Preferably, you would like each cluster to be concentrated on one of the LULC types. For each row, it shows how a certain LULC type is simultaneously captured by different clusters. Ideally, you would prefer a LULC type to be captured by only one cluster. In a very undesirable scenario, each cluster would be capturing multiple LULC types, while each LULC type is captured by multiple clusters. This simply means that your clusters does not capture the LULC patterns at all! Thus you would wish each cluster only captures one LULC type, while leaving no extra pixels for other clusters to capture -- a 100% overlapping between clusters and labels!

Take the row labelled as *clear water* for example, it is nicely captured by one of the cluster leaving no chance for the other clusters. But the cluster seems working properly is also capturing something else shown in its column! Now you may already notice two basic scenarios: (1) determine if a cluster corresponds to a certain LULC type, which is actually interpretation of the clusters in terms of LULC types; and (2) Quantify how good each cluster is capturing a LULC type. The later scenarios further boils down to quantify how much a certain cluster captures/misses, and falsely captures.

We can simply interpret the cluster type as the LULC type with the largest number of pixels captured by the cluster. And this number is referred as the **true positive (TP)**. The cluster may still capture other LULC types, which has been known as the **false positive (FP)** as the cluster falsely captures unintended pixels. If the cluster misses some pixels, the missing pixels are **false negative (FN)** as they shouldn't be treated as unintended pixels. For all those pixels in other clusters that are properly missed by this specific cluster are **true negative (TN)**. Given these four quantities, **TP**, **FP**, **TN**, and **FN**, we are ready to derive several accuracy assessment metrics. Essentially, it will be math operations over the values within the **confusion matrix**.

The very first simple metric is the accuracy for each of the cluster, computed as the **(TP+TN)/total**. Take *clear water* for instance, we treate the cluster capturing the most manually delineated *clear water* pixels. Please figure out how the following is obtained. And repeat over different clusters.

In [ ]:
tp_cw = 5071
tn_cw = 4+1092+15+11+1075+8+710+617+2
total = np.sum(df_confusion.values)  # Simply take the sum of the confusion matrix

acc_cw = (tp_cw+tn_cw)/total
print('The accuracy of clear water cluster is %.5f' % acc_cw)

There are also **precision: TP/(TP+FP)** and **recall: TP/(TP+FN)** to measure how a cluster falsely captures and misses the manually delineated LULC types. As the **precision** also captures how the clustering/classification results present on the ground, it is also known as the **user accuracy (UA)**, whereas the **recall** is also referred as **producer accuracy (PA)** as it measures how things on the ground are missing in the results.

In [ ]:
# Take clear water for example

fp_cw = 3  # FP of clear water cluster
fn_cw = 0  # FN of clear water cluster

UA_cw = tp_cw/(tp_cw+fp_cw)
PA_cw = tp_cw/(tp_cw+fn_cw)

print('The UA and PA of clear water cluster is %.5f  %.5f' % (UA_cw, PA_cw))

As the **UA** and **PA** only cover part of the information, you can also derive the **F1 score** combining both accuracy metrics. The **F1 score** is computed as **F1 = 2/(1/UA+1/PA)**. Similar metric can be the **Jaccard index**. Please see more details [here](https://towardsdatascience.com/common-classification-model-evaluation-metrics-2ba0a7a7436e).

In [ ]:
# F1 score of clear water cluster
f1_cw = 2/(1/UA_cw+1/PA_cw)

print('The F1 score of clear water cluster is %.5f' % f1_cw)

The overall accuracy is simply the total **TP** over the total number of pixels. As mentioned above, we take the LULC type with the largest number of pixels captured by a certain cluster as the interpreted LULC type of this specific cluster.

In [ ]:
acc_total = np.sum(np.max(df_confusion.values, axis=1))/total
print('The total accuracy of the clusters is %.5f' % acc_total)

Again, as you saw, to assess the accuracy of **unsupervised** learning results, we need to interpret the automatically generated patterns, clusters in this case. Then we could be able to evaluate the performance of our algorithm, the **K-means** in this case. We can make this process more automatic by using a small trick, where we search along each row (LULC type) in the **confusion matrix**, and try to find the column with the largest value, and assign the LULC type of the row to the cluster of that column. In this way, we can automatically interpret the clusters. However, please note that, this small trick is already assuming the clusters are capturing each LULC types reasonably well. If each of the known LULC types is not concentrated in one of the clusters, this trick would not work properly.

In [ ]:
# First we need a mapping from both of the LULC codes and cluster indeces to the actual LULC type name.
code_lulc = { 52.0: 'Agriculture',
              11.0: 'Clear water',
              61.0: 'Deciduous forest',
              31.0: 'Residential',
              21.0: 'Sand'}

cluster_lulc = { 0: code_lulc[df_confusion[0].idxmax()],
                 1: code_lulc[df_confusion[1].idxmax()],
                 2: code_lulc[df_confusion[2].idxmax()],
                 3: code_lulc[df_confusion[3].idxmax()],
                 4: code_lulc[df_confusion[4].idxmax()]}

# Now replace the non-intuitive numbers with actual LULC type names and store them into new variables
y_actu2 = y_actu.replace(code_lulc)
y_pred2 = y_pred.replace(cluster_lulc)

In [ ]:
# Create a new confusion matrix and print it out to see how it looks like

df_confusion2 = pd.crosstab(y_actu2, y_pred2)

df_confusion2

<font color='blue'>***Question 0.4: How do you see the numbers reflecting the visualization above in the feature space?***</font>.

Now we have a more properly encoded results as you can see in the above **confusion matrix**. Would you try to assess the clustering results over the other LULC types? As you can imagine, there are indeed nicely implemented modules to compute many kinds of accuracy assessment metrics. The ***sklearn*** is the [module](https://scikit-learn.org/stable/modules/model_evaluation.html) we can use again to achieve such purpose. Please compare the results from the ***sklearn*** and the ones we computed above.

In [ ]:
# Let's import the relevant module and functionailities

from sklearn.metrics import classification_report

print(classification_report(y_actu2, y_pred2))

<font color='green'>***Exercise 0.1: Please do try different band combinations to assess the performance of the K-means with different clustering results.***</font>

So far, we are still working on a small dataset with few thousands of pixels. This would not happen quite often when we work with larger images. And only looking at the data points in the feature space and **confusion matrix** makes it less interesting for inspecting spatial data. Hence, what's next is that we will be working on larger dataset covering a large fraction of the Netherlands, we will again evaluate the performance of the **K-means**, and we will also inspect the results by mapping the clustered pixels back into space for visualization. 

In the previous examples, we knew there were 5 LULC types, and all the pixels are nicely prepared for you in a *.csv* file. This time, given the spatial extent of a large area, there will be potentially several LULC types, and we don't actually know how many beforehand even there are indeed 5 LULC types in those small AOIs. Thus we will have an extra step to find a proper number of clusters, and we will finetune this number further by comparing the results with the known LULC types from the AOIs.

In [ ]:
# Import necessary modules and functionalities

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imread
from sklearn import cluster

# We need to start with loading and stacking the required bands.
# Again, we start with 2 bands that appeared to be useful for the small dataset.
file_list = ['uu_ml/data/b5_2015.TIF', 'uu_ml/data/b6_2015.TIF']  # List to store file names

# You can print to see how the file_list looks like
print(file_list)

# Read the files and stack them together by calling their names
# Use the 'for' loop to iterate over the names to read files
stack = np.array([])  # Empty array to store the stacked images
for file in file_list:
    img = imread(file)  # Read each image file
    print(img.shape)  # Each time, also check the size of the image
    
    # In order to do clustering, image should be reshaped into a single column
    img_col = img.reshape(-1, 1)
    
    # Each time put the reshaped image into the stack
    stack = np.hstack((stack,img_col)) if stack.size else img_col
    # Also to check the size of the stack
    print(stack.shape)

It's time for inspecting the number of clusters for the K-means. We will be using the simple ***elbow curve*** method to see how many clusters will be enough to explain the variance in the dataset. Intuitively, you can have as many clusters as the number of data points, but that won't make sense for "clustering". We want the number to be a lot smaller but still sufficient to capture the patterns in the data. In ***sklearn***, there is a metric called the ***inertia*** measures the average distance from data points to their assigned cluster centroids. The measured value would drop along with increasing number of clusters. The value drops very quickly towards a proper number of clusters that reflects the pattern in the data, once the proper number of clusters is reached, the value drops slower as more clusters would not help to reduce the average distance from data points to designated centroids. Hence, if we plot the values of inertia for different number of clusters, the values will form a curve, where the proper number of clusters would appear as the ***elbow*** of the curve as the dropping trend deflects. Still this method is rough, you may always only use it as a reference, and sometimes you will see multiple deflect points along the curve as multiple clustering patterns would potentially present. You also would like to refer to the real context of your study area when choosing the number of clusters.

In [ ]:
print('This can take quite a while with large images...')

# Inspect some possible number of clusters between 2 and 20.
elbow = []
for k in range(2, 20):
    print('Cluster number = ' + str(k))
    kmean_cluster = cluster.KMeans(n_clusters=k)
    kmean_cluster.fit_predict(stack)
    elbow.append(kmean_cluster.inertia_)

fig = plt.figure(figsize=(5, 5))
plt.plot(range(2, 20), elbow)
plt.grid(True)
plt.title('Elbow curve')

Although an ***elbow*** can be observed around either, 3, 4, or 5 for the number of clusters, we surely would like the number to be larger than 5 to at least capture the known LULC types. It seems that 7 clusters is appropriate. Without hesitation, let's start to evaluate how 7 clusters capture the LULC patterns in the AOIs. To achieve this, we need to rasterize the *.shp* file of the AOIs into pixels that can be compared to the result of the **K-means**. The rasterization will be based upon spatial format of one of the input *.tif* file.

Please note that the ***elbow curve*** is not always the only criteria you may want to use to determine the number of clusters. Instead, you would like to always consider the context of your problem, and also the real world situation related to your problem. In this case, choosing 3 or 4 for the number of clusters would not be meaningful at all as we already knew more types of LULC in our study area, even though the ***elbow*** is indicating these numbers.

In [ ]:
# Create 7 clusters
kmean_cluster = cluster.KMeans(n_clusters=7)
labels = kmean_cluster.fit_predict(stack)

In [ ]:
# Use the rasterio again to rasterize the *.shp file

from rasterio import features

rst = rasterio.open('uu_ml/data/b5_2015.TIF')  # Base image to rasterize the *.shp
meta = rst.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

# Burn the AOIs *.shp file into raster and save it
out_rst = 'uu_ml/data/aoi_rasterized.tif'
with rasterio.open(out_rst, 'w+', **meta) as out:
    out_arr = out.read(1)

    # Create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(aoi.geometry, aoi.aoi_cat))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)


In [ ]:
# Load the rasterized AOIs for comparison

aoi_rst = rasterio.open('data/aoi_rasterized.tif').read(1)

y_actu = pd.Series(list(aoi_rst.reshape(-1,)), name='Manual delineation')
y_pred = pd.Series(list(labels), name='K-means')


# Only keep pixels in the AOIs
y_pred = y_pred[y_actu.index[y_actu.values != 0]]
y_actu = y_actu[y_actu.index[y_actu.values != 0]]

# As before, take a look at the confusion matrix
df_confusion = pd.crosstab(y_actu, y_pred)

df_confusion

<font color='blue'>***Question 0.5: Do you think the number of clusters makes sense to you? Please try to make adjustment, and interpret the meaning of the clusters. Do pay attention to the code mapping below if you changed the number of clusters.***</font>.

In [ ]:
# Do the code mapping again

# First we need a mapping from both of the LULC codes and cluster indeces to the actual LULC type name.
code_lulc = { 52: 'Agriculture',
              11: 'Clear water',
              61: 'Deciduous forest',
              31: 'Residential',
              21: 'Sand'}

cluster_lulc = { 0: code_lulc[df_confusion[0].idxmax()],
                 1: code_lulc[df_confusion[1].idxmax()],
                 2: code_lulc[df_confusion[2].idxmax()],
                 3: code_lulc[df_confusion[3].idxmax()],
                 4: code_lulc[df_confusion[4].idxmax()],
                 6: code_lulc[df_confusion[6].idxmax()]}

# Now replace the non-intuitive numbers with actual LULC type names and store them into new variables
y_actu2 = y_actu.replace(code_lulc)
y_pred2 = y_pred.replace(cluster_lulc)

# Show the LULC coded confusion matrix
df_confusion2 = pd.crosstab(y_actu2, y_pred2)
df_confusion2

In [ ]:
# Evaluate again.

print(classification_report(y_actu2, y_pred2))

As long as you are satisfied by the results, you can alway map the clusters back into space! Let's visulize the clusters from our **K-means** experiment. 

Please note, in order to keep the legend consistent, recall we already have a customized colormap for different LULC types from ***section 0.2***.

In [ ]:
# Assign color codes to LULC types 
symbology = {'Agriculture': cols[5],
             'Clear water': cols[20],
             'Deciduous forest': cols[3],
             'Residential': cols[17],
             'Sand': cols[11]}

# Visualize
fig1,(ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
show(b5_2020, ax=ax1, cmap='gray', alpha=0.25)
aoi.plot(ax=ax1, column='land_cover', legend=True, color=aoi['land_cover'].map(symbology))

from matplotlib.lines import Line2D
custom_points = [Line2D([0], [0], marker="o", linestyle="none", markersize=5, color=color) for color in symbology.values()]
leg_points = ax1.legend(custom_points, symbology.keys(), loc='upper right', frameon=False)
ax1.add_artist(leg_points)

# Assign color codes to LULC types 
symbology2 = {0: cols[17],
             1: cols[5],
             2: cols[20],
             3: cols[11],
             4: cols[3],
             5: cols[9],
             6: cols[9]}

# Visualize
# Because the predicted labels are still in one column, you need to reshape it back to original image shape
row, col = img.shape  # Get the original dimensions of the image

cm = ListedColormap(symbology2.values())
imin = min(symbology2)  # Colormap range
imax = max(symbology2)

print('Printing large image takes time...')
ax2.imshow(labels.reshape(row, col), cmap=cm, interpolation='none', vmin=imin, vmax=imax)

<font color='green'>***Exercise 0.2: Are you satisfied with the results? It's your role to continue explore different combinations of input bands for the K-means clustering. Try to evaluate the accuracy with 2, 3 or more bands. Also visualize the final maps, and discuss with your classmates about the maps. When you repeat the entire procedure, you are highly encouraged to compile a single piece of repeatable program, it can be a fraction of code above in section 0.3. You are also encouraged to record the results for different scenarios of clutering, for instance, plot how accuracy changes with the changing band combination, number of bands, clusters, etc.***</font>

*Start to load image bands you would like to use:*

```
file_list = ['uu_ml/data/b5_2015.TIF', 'uu_ml/data/b6_2015.TIF', ..., ...,]
```

*Also load your labelled AOIs, and rasterize it:*

```
aoi = gpd.read_file('...')

rst = rasterio.open('...')  # Base image to rasterize the *.shp
meta = rst.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

...
```

*Clustering and inspect the number of clusters with different evaluations methods. Some example can be found at: 
https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a
or
https://www.analyticsvidhya.com/blog/2021/05/k-mean-getting-the-optimal-number-of-clusters/
You may still want to try the "elbow" method?*

```
elbow = []
for k in range(2, 20):
    ...
    ...
```

*Evaluate the accuracy with different number of clusters and bands used*

```
...
...
```